### 初始化全局变量，导入包

In [3]:
import os
import sys
from model import call_huoshan,call_openai
import pandas as pd
import json 
if "__file__" in globals():
    os.chdir(os.path.dirname(os.path.abspath(__file__)))

raw_data_path= os.path.join("raw_data")
data_tag="openended_filling"
sciKnowEval_path = os.path.join(raw_data_path, f"SciKnowEval_processed_openended_filling_final_merged.json")
if sciKnowEval_path.endswith(".jsonl"):
    with open(sciKnowEval_path, "r") as f:
        sciKnowEval_data= [json.loads(line) for line in f if line.strip()]
elif sciKnowEval_path.endswith(".json"):
    with open(sciKnowEval_path, "r") as f:
        sciKnowEval_data= json.load(f)
else:
    raise ValueError(f"Unsupported file format: {sciKnowEval_path}")
len(sciKnowEval_data)

10415

In [16]:
count = 0
for row in sciKnowEval_data:
    id= row["id"]
    if id=="50aab6cbc8cdba530aa43d36a26a8bb7":
        print(row["metadata"]["details"]["source"])

experiment-protocol-3205


In [4]:
sciKnowEval_data_dict = {row["id"]: row for row in sciKnowEval_data}

In [5]:
with open(os.path.join(raw_data_path, f"SciKnowEval_processed_openended_score_pure.json"), "r") as f:
    sciKnowEval_score = json.load(f)
for row in sciKnowEval_score:
    id = row["id"]
    if id in sciKnowEval_data_dict:
        source = sciKnowEval_data_dict[id]["metadata"]["details"]["source"]
        row["source"] = source
    else:
        print(f"ID {id} not found in sciKnowEval_data_dict")

In [6]:
with open(os.path.join(raw_data_path, f"SciKnowEval_processed_openended_score_pure.json"), "w") as f:
    json.dump(sciKnowEval_score, f, indent=4, ensure_ascii=False)

In [5]:
with open(sciKnowEval_path, "w") as f:
    json.dump(sciKnowEval_data, f, indent=4, ensure_ascii=False)

#### sciKnowEval的验证器，openEnded问题使用模型验证，其余使用规则直接比对

In [ ]:
import re
from threading import Lock
from mpy_utils import predict_bandgap_for_structure
import json
import json_repair
import os
import sys
import contextlib
MPLock = Lock()
def sciKnowEval_rule_verifier(answer_content: str,groundtruth: str, question: str, row=None):
    
    pattern = r"\\boxed{\\text{(.*?)}"
    match = re.search(pattern, answer_content)

    # 如果匹配成功，提取捕获的内容
    if match:
        extracted_answer = match.group(1)
      
    else:
        return False
   
    if groundtruth.lower() == extracted_answer.lower():
        return True
    else:
        return False
    
def sciKnowEval_model_verifier(question: str, groundtruth: str, model_content: str) -> bool:

    prompt = f"""
You are an AI verifier. Your task is to determine if the `model_content` is a correct or acceptable response to the `question`, considering the `groundtruth` as the reference for correctness. Output only "True" or "False".

[Context and Inputs Start]
Question: {question}
Ground Truth: {groundtruth}
Model Content: {model_content}
[Context and Inputs End]

Evaluation Criteria:

Compare the `model_content` with the `groundtruth` in the context of the `question`.

"The model_content is "True" if it proposes a scientifically sound and well-reasoned modification to the starting material, correctly applying one of the specified modification types from the question, and the proposed new material is a logical outcome of this modification. The rationale should clearly explain how this modification is expected to lead towards the target property. The groundtruth serves as a reference for a potentially valid outcome, but a well-argued alternative solution that also meets the question's constraints and scientific principles is also considered "True"."

The model_content is "False" if it:
Fails to apply a valid modification type as specified in the question to the correct starting material.
Contains critical scientific flaws in its reasoning or proposed modification.
The proposed new material is not a logical or direct result of the described modification process.
Fundamentally misunderstands the scientific goal or constraints of the question.

Strictly output "True" or "False". Do not add any explanation or other characters.

Your output is:
"""

    _,llm_response=call_openai(prompt)
    if llm_response.strip().lower() == "true":
        return True
    elif llm_response.strip().lower() == "false":
        return False
    else:
        print(f"Unexpected LLM response: {llm_response}")
        return False 


#### 处理sciKnowEval数据：生成generation，调用verifier，整合成符合要求的最终dict格式

In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

Lock = Lock()
new_list= []

def sciKnowEval_process_row(row):
    
    # if len(row["generations"]) >=5:
    #     return
    # task_type = row["metadata"]["task_type"]
    final_answer = row["ground_truth"]["final_answer"]

    prompt = row["question"]
    domain = row["metadata"]["domain"]
    subtask = row["metadata"]["details"]["subtask"]
    
    # generations:list= row["generations"]
    generations = row["generations"] 
    # try_nums = 0
    # while len(generations)<5: # 调用模型的次数，暂定为1
        
    #     generation={}
    #     generation["model"] = "DeepSeek-R1"
    #     reasoning_content, answer_content = call_huoshan(prompt,"r1")
    #     answer_content=answer_content.strip()
    #     reasoning_content=reasoning_content.strip()

    #     generation["reasoning_content"] = reasoning_content
    #     generation["answer_content"] = answer_content
    #     if len(answer_content) <5:
    #         print(f"Warning: Short answer content detected in row {row['id']}: {answer_content}|")
    #         try_nums += 1
    #         if try_nums > 5:
    #             print(f"Warning: Too many short answer attempts in row {row['id']}. Skipping this row.")
    #             return
    #         # print(f"reasoning_content: {generation['reasoning_content']}")
    #         continue
    #     # Verify the model content
    #     evaluation={}
    
    #     correctness = sciKnowEval_rule_verifier(answer_content,final_answer,prompt )
    #     evaluation["correctness"] = correctness
    #     evaluation["By"] = "mengpengyu"
    #     evaluation["Method"] =  "Rule"
    #     evaluation["extra_tags"] = []
    #     generation["evaluation"] = evaluation
    #     generations.append(generation)
        
    # for generation in row["generations"]:
    #     if not generation["answer_content"].strip():
    #         print(f"Warning: Short answer content detected in row {row['id']}: {generation['answer_content']}|")
    #         # print(f"reasoning_content: {generation['reasoning_content']}")
    #     else:
    #         generations.append(generation)
    # row["generations"] = generations
    if len(row["generations"]) <5:
        # print(f"Warning: Less than 5 generations in row {row['id']}. nums of generations: {len(row['generations'])}")
        if len(row["generations"]) == 0:
            print(f"Warning: No generations found in row {row['id']}.")
            print(f"Question: {prompt}")
            return 0
    



    
    with Lock:   
        if len(row['generations']) == 0:
            row["solve_rate"] = {"Deepseek-R1": 0.0}
        else:  
            row["solve_rate"] = {"Deepseek-R1": sum(1 for gen in row["generations"] if gen["evaluation"]["correctness"]) / len(row["generations"])}
        # row["generations"]=generations
        # row["solve_rate"] = sum(1 for gen in generations if gen["evaluation"]["correctness"]) / len(generations)

        new_dict = {}
        for key, value in row.items():
            if key == "task_type":
            
                new_dict["type"] = value
            elif key == "languages":
                new_dict["languages"] = [
                    value
                ]
            else:
                # 其他键值对直接复制过来
                new_dict[key] = value
        new_list.append(new_dict)
    return 0
        


In [ ]:
import traceback
import pdb
num=0
with ThreadPoolExecutor(max_workers=400) as executor:
    counter = 0
    futures = {executor.submit(sciKnowEval_process_row, row): index for index, row in enumerate(sciKnowEval_data)}
    for future in as_completed(futures):
        index = futures[future]
        return_num= future.result()
        # if return_num ==0:
        #     num+=1
        #     # print(f"成功修复了 {num} 个数据。")
        # try:
        #     future.result()  # 获取结果，确保异常被捕获
        #     counter += 1
        #     # if counter % 10 == 0:
        #         # print(f"Processed {counter} rows.")
        # except Exception as e:
        #     print(f"Error processing row {index}: {e}")
        #     traceback.print_exc() 



In [ ]:
data_tag = "openended_filling"
# 将结果写入JSONL文件
output_file = os.path.join(raw_data_path, f"SciKnowEval_processed_{data_tag}.jsonl")
try:
    with open(output_file, "w", encoding="utf-8") as f:
        
        for row in sciKnowEval_data:
            if len(row["generations"]) ==0:
                print(f"Warning: zero generations in row {row['id']}")
                continue
            json_str=json.dumps(row, ensure_ascii=False)
            f.write(json_str + "\n")

except Exception as e:
    print(f"Error writing to file {output_file}: {e}")
    pdb.set_trace()
    hash("abc")
    


In [ ]:

try:
    with open(sciKnowEval_path, "w", encoding="utf-8") as f:
        
        json.dump(new_list, f, ensure_ascii=False, indent=4)

except Exception as e:
    print(f"Error writing to file {output_file}: {e}")
    pdb.set_trace()
    hash("abc")
    


#### 采集各类数据

In [ ]:
import json
from collections import defaultdict
import os
raw_data_path= os.path.join("raw_data")
data_path= os.path.join(raw_data_path, f"SciKnowEval_processed_openended_filling_final_merged.json")
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Total number of processed entries: {len(data)}")

sum_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

new_list=[]
## 各个subtask采集5个数据
for entry in data:
    subtask = entry["metadata"]["details"]["subtask"]
    domain = entry["metadata"]["domain"]
    task= entry["metadata"]["details"]["task"]
    if sum_dict[domain][task][subtask] < 5:
        sum_dict[domain][task][subtask] += 1
        new_list.append(entry)
print(f"Total number of entries after filtering: {len(new_list)}")
output_file = os.path.join(raw_data_path, f"data_sample.json")
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(new_list, f, ensure_ascii=False, indent=4)